In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('train.csv')

In [6]:
df.shape

(404290, 6)

In [8]:
df.tail()

,id,qid1,qid2,question1,question2,is_duplicate
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0
404289,404289,537932,537933,What is like to have sex with cousin?,What is it like to have sex with your cousin?,0


In [9]:
new_df = df.sample(30000)

In [10]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [11]:
new_df.duplicated().sum()

0

In [12]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
323922,Does anyone want to share their experience abo...,How does it feel to travel alone?
223807,What are some interesting ways in which certai...,What was the first living organism on Earth? W...
388712,What could be the possible reasons for Nitish ...,I want to why the pension component will not b...
117361,How do I log in my Facebook account if you for...,I forget my Facebook account password and I al...
175434,How is the pH of nitric acid determined?,How is the pH of acetylsalicylic acid determined?


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [14]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [15]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
323922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223807,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355312,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
296612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
204016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
265467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [17]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
323922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223807,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
175434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7388333333333333

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7255